In [1]:
from gemmavq_whisper import GemmaWhisperForConditionalGeneration
from transformers import AutoFeatureExtractor, AutoTokenizer, AutoProcessor
import librosa
import torch

torch.set_grad_enabled(False)

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2025-08-01 14:36:29.617476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754058989.627550  122991 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754058989.631977  122991 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register facto

In [2]:
GemmaWhisperForConditionalGeneration.register_for_auto_class("AutoModel")

In [3]:
!ls gemma3n-audio-vq-whisper-decoder-v5

checkpoint-147500  checkpoint-148000  checkpoint-148500  runs


In [4]:
model = GemmaWhisperForConditionalGeneration.from_pretrained(
    'gemma3n-audio-vq-whisper-decoder-v5/checkpoint-148500', torch_dtype = 'auto')

In [5]:
_ = model.cuda()

In [6]:
(model.model.decoder.embed_tokens.weight == model.proj_out.weight).float().mean()

tensor(1., device='cuda:0')

In [7]:
feature_extractor = AutoFeatureExtractor.from_pretrained('mesolitica/gemma-3n-e4b-it-audio-encoder')
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3-turbo')

In [8]:
from streaming import MDSWriter, LocalDataset

dataset = LocalDataset('mosaic-stt-include-malaysian')

In [9]:
encoder = model.model.get_encoder()

In [10]:
y, sr = librosa.load('common_voice_ba_26517811.mp3', sr = feature_extractor.sampling_rate)
features = feature_extractor([y], return_tensors = 'pt')
features['input_features'] = features['input_features'].cuda()
features['input_features_mask'] = features['input_features_mask'].cuda()
_, tokens = encoder(**features)
print(tokens)

tensor([30775,  1587, 15003, 31780,    58, 16102, 25962,  3368, 24676, 28593,
         4429, 22812,  5879, 28407, 22852, 26137, 12420, 18597, 30501, 19939,
         5266, 21355,  1644, 23031,  3164,  4653, 30055, 11051, 20250],
       device='cuda:0')


In [11]:
!ls *.mp3

218757.mp3  common_voice_ba_26517811.mp3


In [15]:
y, sr = librosa.load('218757.mp3', sr = feature_extractor.sampling_rate)
input_ids = tokenizer(
    '<|startoftranscript|><|ru|><|transcribe|><|notimestamps|>', 
    add_special_tokens = False, return_tensors = 'pt')['input_ids']
features = feature_extractor([y], return_tensors = 'pt')
features['input_features'] = features['input_features'].cuda()
features['input_features_mask'] = features['input_features_mask'].cuda()
features['attention_mask'] = features['input_features_mask']
features['decoder_input_ids'] = input_ids.cuda()

generate_kwargs = dict(
    **features,
    max_new_tokens=1024,
)
generation_output = model.generate(**generate_kwargs)
tokenizer.decode(generation_output[0])

'<|startoftranscript|><|ms|><|transcribe|><|notimestamps|> Sikit sebelum ini membuka pelikannya kebenaran khas kebenaran terhadap pihak menengah tujuan diwarisi, betul-betul dihantar untuk dikunci orang. Padahal saya lihat kata-kata boleh ambil risau kalau saya lebih buruk mengatakan dokumen di rumah.<|endoftext|>'

In [16]:
model.push_to_hub('mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder')

Processing Files (0 / 0)                : |                                                                                                                                   |  0.00B /  0.00B            
New Data Upload                         : |                                                                                                                                   |  0.00B /  0.00B            

  /tmp/tmpkd27c6j1/model.safetensors    :   0%|                                                                                                                               | 24.3kB / 3.61GB            

Processing Files (0 / 1)                :   0%|                                                                                                                               | 24.3kB / 3.61GB,   ???B/s  

Processing Files (0 / 1)                :   2%|██▎                                                                                                                            | 67.3M

Processing Files (0 / 1)                :  84%|███████████████████████████████████████████████████████████████████████████████████████████████████████████                    | 3.04GB / 3.61GB,  692MB/s  

Processing Files (0 / 1)                :  89%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉              | 3.21GB / 3.61GB,  698MB/s  

Processing Files (0 / 1)                :  94%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊        | 3.38GB / 3.61GB,  704MB/s  

Processing Files (0 / 1)                :  98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉   | 3.52GB / 3.61GB,  705MB/s  

Processing Files (0 / 1)                :  98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████  | 3.56

CommitInfo(commit_url='https://huggingface.co/mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder/commit/00fff100c20860154df96984ea0bdb68edc428b2', commit_message='Upload GemmaWhisperForConditionalGeneration', commit_description='', oid='00fff100c20860154df96984ea0bdb68edc428b2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder'), pr_revision=None, pr_num=None)

In [17]:
tokenizer.push_to_hub('mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder')

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder/commit/00fff100c20860154df96984ea0bdb68edc428b2', commit_message='Upload tokenizer', commit_description='', oid='00fff100c20860154df96984ea0bdb68edc428b2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder'), pr_revision=None, pr_num=None)

In [18]:
feature_extractor.push_to_hub('mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder')

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder/commit/00fff100c20860154df96984ea0bdb68edc428b2', commit_message='Upload feature extractor', commit_description='', oid='00fff100c20860154df96984ea0bdb68edc428b2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder', endpoint='https://huggingface.co', repo_type='model', repo_id='mesolitica/gemma3n-audio-encoder-VQ-whisper-decoder'), pr_revision=None, pr_num=None)